In [37]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import urllib.parse

In [38]:
job_title = [] # css-o171kl
company = [] # css-17s97q8
location = [] # css-5wys0k
type_of_employment = [] # css-n2jc4m
description = [] # css-o171kl
country = []

In [39]:
def generate_url(query, page_number):
    params = {'a': 'hpd', 'q': query, 'start': page_number}
    return "https://wuzzuf.net/search/jobs/?{}".format(urllib.parse.urlencode(params))

In [40]:
def web_scrap(query):
    page = 0
    while True:
        response = requests.get(generate_url(query, page))

        soup = BeautifulSoup(response.content, "html5lib")

        #Store all the jobs in the jobs array
        jobs = []
        for s in soup.find_all(class_="css-1gatmva e1v1l3u10"):
            jobs += s

        if len(jobs) == 0:
            break

        for i in range(1, len(jobs)):
            # find the job title
            title_desc = jobs[i].find_all(class_="css-o171kl")

            #find the description
            desc = ""
            for j in range(0, len(title_desc)):
                # the first one is the job title.
                if j == 0: 
                    job_title.append(title_desc[j].text)
                # if this is the last description for the job don't pt ","
                elif j == len(title_desc)-1:
                    desc += title_desc[j].text
                # to put the description and ","
                else:
                    desc += title_desc[j].text
                    desc += ", "

            # append the description
            description.append(desc)

            # find all the details
            comp = jobs[i].find_all(class_="css-17s97q8")
            loc = jobs[i].find_all(class_="css-5wys0k")
            type_emp = jobs[i].find_all(class_="css-n2jc4m")

            # find all employment types
            all_types = ""
            for j in range(0, len(type_emp)):
                if j == len(type_emp)-1:
                    all_types += type_emp[j].span.text
                else:
                    all_types += type_emp[j].span.text
                    all_types += ", "

            # append all the details
            company.append(comp[0].text)
            location.append(loc[0].text)
            type_of_employment.append(all_types)

        page += 1

In [41]:
web_scrap("devops")

<h3>Extract the country from the location array: </h3>

In [42]:
for i in range(0, len(location)):
    # extrat the country from the location
    coun = location[i].split(',')[-1]
    country.append(coun)
    # to remove the country from the location
    location[i] = location[i].partition(coun)[0]
    # to remove the "," at the last of the location
    location[i] = location[i][:len(location[i])-1]

In [43]:
dic = {
    "Job Title": job_title,
    "Company Name: ": company,
    "Location": location,
    "Country":country,
    "Employment Type":type_of_employment,
    "Description":description
}

In [44]:
df = pd.DataFrame(dic)
df

,Job Title,Company Name:,Location,Country,Employment Type,Description
0,DevOps/ProdOPS Engineer,Cegedim -,"New Cairo, Cairo",Egypt,Full Time,"Experienced, · IT/Software Development, · En..."
1,DevOps/ ProdOPS Engineer,Cegedim -,"New Cairo, Cairo",Egypt,Full Time,"Experienced, · IT/Software Development, · En..."
2,Senior Devops Engineer,Confidential -,"New Cairo, Cairo",Egypt,Full Time,"Experienced, · IT/Software Development, · En..."
3,DevOps Java Developer,Ejada (Cairo) -,Cairo,Egypt,Full Time,"Experienced, · IT/Software Development, · En..."
4,Javascript Developer with Devops exp - Remote,GetTechForce.com -,Cairo,Egypt,"Full Time, Work From Home","Experienced, · IT/Software Development, · En..."
...,...,...,...,...,...,...
126,.NET & SharePoint Developer,BlueCloud -,"Sheraton, Cairo",Egypt,Full Time,"Experienced, · Installation/Maintenance/Repai..."
127,Backend Developer,Profolio Consulting -,Seattle,United States,"Full Time, Work From Home","Experienced, · IT/Software Development, · En..."
128,Software Engineering Manager,3D Diagnostix -,"Maadi, Cairo",Egypt,"Full Time, Work From Home","Manager, · IT/Software Development, · Engine..."
129,Magento Architect,Dal Digital - Saudi Arabia -,Cairo,Egypt,Full Time,"Experienced, · IT/Software Development, · En..."
